<a href="https://colab.research.google.com/github/AndrewRomitti/Swida/blob/main/PTGradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andrewromitti123","key":"faac1459af0bc94cd1618c689bab8fd0"}'}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#!kaggle datasets download -d hamdallak/the-iqothnccd-lung-cancer-dataset
!kaggle datasets download -d kabil007/lungcancer4types-imagedataset

Dataset URL: https://www.kaggle.com/datasets/kabil007/lungcancer4types-imagedataset
License(s): apache-2.0
 92% 109M/119M [00:00<00:00, 150MB/s] 
100% 119M/119M [00:00<00:00, 152MB/s]


In [ ]:
#!unzip /content/the-iqothnccd-lung-cancer-dataset.zip > /dev/null 2>&1
!unzip /content/lungcancer4types-imagedataset.zip > /dev/null 2>&1

# For Finetuning of the Resnet,

Credit: https://github.com/Spandan-Madan/Pytorch_fine_tuning_Tutorial

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224), # Center cropping to a consistent size
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
import os

data_dir = r'/content/Data'
BATCH_SIZE = 16

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train','valid']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'valid']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}
dset_classes = dsets['train'].classes

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 10 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.


# DATASET INFO
NUM_CLASSES = 4 # set the number of classes in your dataset

# DATALOADER PROPERTIES
BATCH_SIZE = 16 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 0 # set to 1 if want to run on gpu.

In [ ]:
use_gpu = True
if use_gpu:
    torch.cuda.set_device(0)

count=0

In [ ]:
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            running_loss = 0.0
            running_corrects = 0

            counter=0
            # Iterate over data.
            for data in dset_loaders[phase]:
                inputs, labels = data
                # wrap them in Variable
                if use_gpu:
                    inputs = inputs.float().cuda()
                    labels = labels.long().cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                inputs, labels = Variable(inputs), Variable(labels)
                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)

                loss = criterion(outputs, labels)
                # print('loss done')
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                # if counter%10==0:
                #     print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    # print('loss backward')
                    loss.backward()
                    # print('done loss backward')
                    optimizer.step()
                    # print('done optim')
                # print evaluation statistics
                try:
                    # running_loss += loss.data[0]
                    running_loss += loss.item()
                    # print(labels.data)
                    # print(preds)
                    running_corrects += torch.sum(preds == labels.data)
                    # print('running correct =',running_corrects)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            print('trying epoch loss')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            # deep copy the model
            if phase == 'valid':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

# This function changes the learning rate over the training model.
def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, NUM_CLASSES)


criterion = nn.CrossEntropyLoss()

if use_gpu:
    criterion.cuda()
    model_ft.cuda()

optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=0.001)



# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/49
----------
LR is set to 0.001


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


trying epoch loss
train Loss: 0.1021 Acc: 0.4095
trying epoch loss
valid Loss: 0.4137 Acc: 0.3194
new best accuracy =  0.3194444444444444
Epoch 1/49
----------
trying epoch loss
train Loss: 0.0713 Acc: 0.4535
trying epoch loss
valid Loss: 0.0741 Acc: 0.3889
new best accuracy =  0.3888888888888889
Epoch 2/49
----------
trying epoch loss
train Loss: 0.0606 Acc: 0.5563
trying epoch loss
valid Loss: 0.0711 Acc: 0.3889
Epoch 3/49
----------
trying epoch loss
train Loss: 0.0595 Acc: 0.5612
trying epoch loss
valid Loss: 0.0659 Acc: 0.5556
new best accuracy =  0.5555555555555556
Epoch 4/49
----------
trying epoch loss
train Loss: 0.0574 Acc: 0.5546
trying epoch loss
valid Loss: 0.0837 Acc: 0.4028
Epoch 5/49
----------
trying epoch loss
train Loss: 0.0575 Acc: 0.5661
trying epoch loss
valid Loss: 0.0731 Acc: 0.5556
Epoch 6/49
----------
trying epoch loss
train Loss: 0.0522 Acc: 0.6101
trying epoch loss
valid Loss: 0.0841 Acc: 0.4722
Epoch 7/49
----------
trying epoch loss
train Loss: 0.0491 Acc

Exception ignored in: <function _ConnectionBase.__del__ at 0x791028894550>
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 250, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/

trying epoch loss
valid Loss: 0.0785 Acc: 0.6250
Epoch 18/49
----------
trying epoch loss
train Loss: 0.0185 Acc: 0.9005
trying epoch loss
valid Loss: 0.0858 Acc: 0.6250
Epoch 19/49
----------
trying epoch loss
train Loss: 0.0173 Acc: 0.8923
trying epoch loss
valid Loss: 0.0773 Acc: 0.5972
Epoch 20/49
----------
LR is set to 1.0000000000000003e-05
trying epoch loss
train Loss: 0.0156 Acc: 0.9250
trying epoch loss
valid Loss: 0.0510 Acc: 0.7361
Epoch 21/49
----------
trying epoch loss
train Loss: 0.0150 Acc: 0.9135
trying epoch loss
valid Loss: 0.0644 Acc: 0.6944
Epoch 22/49
----------
trying epoch loss
train Loss: 0.0132 Acc: 0.9347
trying epoch loss
valid Loss: 0.0591 Acc: 0.7778
new best accuracy =  0.7777777777777778
Epoch 23/49
----------
trying epoch loss
train Loss: 0.0149 Acc: 0.9315
trying epoch loss
valid Loss: 0.0556 Acc: 0.7778
Epoch 24/49
----------
trying epoch loss
train Loss: 0.0122 Acc: 0.9380
trying epoch loss
valid Loss: 0.0544 Acc: 0.7917
new best accuracy =  0.79166

In [ ]:
torch.save(model_ft.state_dict(),'/content/model_ft_3.pt')

# GradCam

In [ ]:
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import torch.nn as nn
from torchvision import datasets, models, transforms

In [ ]:
model = resnet50(pretrained=True)
#model = models.VGG16(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

state_dict = torch.load('/content/gdrive/MyDrive/model_ft_3.pt')
model.load_state_dict(state_dict)

target_layers = [model.layer4[-1]]
#target_layers = model.features[-1]

<ipython-input-165-c863a6f4549d>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/content/gdrive/MyDrive/model_ft_3.pt')


In [ ]:
from PIL import Image


#image = Image.open('/content/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000005 (9).png')
#image = Image.open('/content/Data/train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/000002 (6).png')
image = Image.open('/content/Data/train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/000013 (6).png')
image = image.convert('RGB')
rgb_img = image.resize((224, 224))

rgb_img = np.array(rgb_img).astype(np.float32) / 255.0

transform1 = transforms.Resize((224,224))
transform2 = transforms.ToTensor()
input_tensor = transform1(image)
input_tensor = transform2(input_tensor).cpu().unsqueeze(0)

min_val = input_tensor.min()
max_val = input_tensor.max()
normalized_tensor = (input_tensor - min_val) / (max_val - min_val)
input_tensor = normalized_tensor.type(torch.float32)

#numpy_image = transform(image).cpu().numpy()

#input_tensor = (numpy_image - numpy_image.min()) / (numpy_image.max() - numpy_image.min())

print(input_tensor.shape)
print(input_tensor.min())
print(input_tensor.max())
print(input_tensor.dtype)

torch.Size([1, 3, 224, 224])
tensor(0.)
tensor(1.)
torch.float32


In [ ]:
print(dsets['valid'].classes)

['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


In [ ]:
from PIL import Image

targets = [ClassifierOutputTarget(2)]

with GradCAMPlusPlus(model=model, target_layers=target_layers) as cam:
  # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
  grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
  # You can also get the model outputs without having to redo inference
  model_outputs = cam.outputs

In [ ]:
img = Image.fromarray(visualization)
img.save('cam.png')